In [ ]:
import numpy as np
import pandas as pd
import pathlib
from tqdm.notebook import tqdm

In [ ]:
base_path = "/home/jakobs"

In [ ]:
endpoints_md = pd.read_csv(f"{base_path}/BiHealth/onnx/endpoints.csv").drop(columns="Unnamed: 0").set_index("endpoint")#[["endpoint", "eligable", "n", "freq", "phecode", "phecode_string", "phecode_category", "sex"]]
endpoints_md

In [ ]:
import math
import warnings
from socket import gethostname

import numpy as np
import torch
import torchvision as tv
import torchmetrics

from retinalrisk.models.supervised import (
    ImageTraining
)
from retinalrisk.modules.head import MLPHead

def setup_training():
    
    def get_head(num_head_features, num_endpoints):

        cls = MLPHead

        return cls(
            num_head_features,
            num_endpoints,
            incidence=None,
            dropout=0.5,
            gradient_checkpointing=False,
            num_hidden =512, 
            num_layers =2,
            #loss=None,
        )
    
    base_path = "/home/jakobs"
    x = torch.load(f"{base_path}/BiHealth/ckpts/ckpt_partition_4")
    losses = x['hyper_parameters']["losses"]
    label_mapping = x['hyper_parameters']["label_mapping"]
    incidence_mapping = x['hyper_parameters']["incidence_mapping"]

    #encoder = tv.models.__dict__[args.model.encoder](pretrained=args.model.pretrained)
    weights = 'DEFAULT' 
    encoder = tv.models.__dict__["convnext_small"](weights=weights) 
    #print(encoder)

    outshape = 768 

    setattr(encoder.classifier, '2', torch.nn.Identity())

    head = get_head(num_head_features=768, num_endpoints = 1171)

    model = ImageTraining(encoder=encoder, head=head, losses=None, label_mapping=label_mapping, incidence_mapping=None, task="tte")

    return model

In [ ]:
import torch.nn as nn
import PIL
from typing import Union
from random import choice
import torchvision as tv
import torchvision.transforms.functional as TF
from torchvision.transforms import InterpolationMode

import torch
import torchvision.transforms as transforms
from skimage.io import imread

crop_ratio = [0.66]
img_size_to_gpu = 420 # 420

class AdaptiveRandomCropTransform(nn.Module):
    def __init__(self,
                 crop_ratio: Union[list,float],
                 out_size: int,
                 interpolation=InterpolationMode.BILINEAR):
        super().__init__()
        self.crop_ratio = crop_ratio
        self.out_size = out_size
        self.interpolation = interpolation

    def forward(self, sample):
        input_size = min(sample.size)
        if isinstance(self.crop_ratio, list):
            crop_ratio = choice(self.crop_ratio)
        else:
            crop_ratio = self.crop_ratio

        crop_size = int(crop_ratio * input_size)
        if crop_size < self.out_size:
            crop_size = tv.transforms.transforms._setup_size(self.out_size,
                                                             error_msg="Please provide only two dimensions (h, w) for size.")
            i, j, h, w = transforms.RandomCrop.get_params(sample, crop_size)
            return TF.crop(sample, i, j, h, w)
        else:
            crop_size = tv.transforms.transforms._setup_size(crop_size,
                                                             error_msg="Please provide only two dimensions (h, w) for size.")
            i, j, h, w = transforms.RandomCrop.get_params(sample, crop_size)
            cropped = TF.crop(sample, i, j, h, w)
        out = TF.resize(cropped, self.out_size, self.interpolation)
                        
        return out
    

In [ ]:
# Define the transforms to apply
transform = transforms.Compose([
    AdaptiveRandomCropTransform(crop_ratio=crop_ratio,
                                out_size=img_size_to_gpu,
                                interpolation=InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.CenterCrop(size=384),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
                    
])

#invTrans = transforms.Compose([ 
#    transforms.Normalize(mean = [ 0., 0., 0. ], std = [ 1/0.229, 1/0.224, 1/0.225 ]), 
#    transforms.Normalize(mean = [ -0.485, -0.456, -0.406],  std = [ 1., 1., 1. ])])

In [ ]:
from torch.utils.data import Dataset, DataLoader

class EPICImagesDataset(Dataset):
    def __init__(self, data_images, transform):
        self.data_images = data_images
        self.transform = transform

    def __len__(self):
        return len(self.data_images)

    def __getitem__(self, index):
        img_name = self.data_images.iloc[index]["distfilename"]

        try:
            img_np = imread(f"{base_path}/BiHealth/Data/EPICImages/{img_name}")
        except:
            img_np = imread(f"{base_path}/BiHealth/Data/EPICImages_PoorQuality/{img_name}")

        img_pil = PIL.Image.fromarray(img_np)

        try:
            img_tensor = self.transform(img_pil)
        except:
            print(img_name)

        return img_name, img_tensor

def collate_fn(batch):
    img_names, img_tensors = zip(*batch)
    img_names = list(img_names)
    img_tensors = torch.stack(img_tensors)
    return img_names, img_tensors

def predict_batch(model, img_batch):
    loghs = model(img_batch)["head_outputs"]["logits"].detach().cpu().numpy()
    return loghs

corrupted_files = ["0AIULA8E31FVNEXW_epiceye07142.png", 
                   "0AIULA8E315UZ3KA_epiceye03519.png", 
                   "0AIULA8E3354WXMB_epiceye03739.png",
                   "0AIULA8E32JU9I3E_epiceye00148.png",
                   "0AIULA8E3354JOZ0_epiceye06941.png",
                  "0AIULA8E315XYVCL_epiceye05788.png",
                   "0AIULA8E315WMUA2_epiceye05000.png",
                  "0AIULA8E32JRQLLF_epiceye01039.png",
                  "0AIULA8E329S6BCD_epiceye02546.png",
                  "0AIULA8E31RCZB57_epiceye00155.png",
                  "0AIULA8E31REEET3_epiceye05711.png",
                   "0AIULA8E32SJJSL9_epiceye03063.png",
                  "0AIULA8E31FQZ4OO_epiceye06657.png",
                   "0AIULA8E1HIF7IFB_epiceye05427.png",
                  "0AIULA8E32SDCBXM_epiceye00289.png",
                  "0AIULA8E1KEKKV8Y_epiceye05179.png"]

data_images = pd.read_stata(f"{base_path}/BiHealth/Data/StudyData/BiHealth_20230313_Long.dta").query("distfilename!=@corrupted_files")
dataset = EPICImagesDataset(data_images, transform)
#dataset = EPICImagesDataset(data_images, transform, base_path, num_workers=4, cache_size=100)
dataloader = DataLoader(dataset, batch_size=5, shuffle=False, num_workers=32, collate_fn=collate_fn, drop_last=False)
model = setup_training()

In [ ]:
partitions = [4]

metadata = []
for iteration in tqdm(range(10)):
    for partition in tqdm(partitions):
        with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
            ckpt = torch.load(f"{base_path}/BiHealth/ckpts/ckpt_partition_{partition}")
            model.encoder.load_state_dict({k[8:]:v for k, v in ckpt["state_dict"].items() if "encoder" in k}, strict=True)
            model.head.load_state_dict({k[5:]:v for k, v in ckpt["state_dict"].items() if "head" in k}, strict=True)
            model.eval();
            model.to("cuda")
            # instantiate cktp here
            for img_names, img_batch in tqdm(dataloader):
                img_batch = img_batch.to("cuda")
                loghs = predict_batch(model, img_batch)
                for img_name, logh in zip(img_names, loghs):
                    metadata.append({"partition": partition, "img_name": img_name, "iteration": iteration, "loghs": logh})
                del img_batch

metadata_df = pd.DataFrame(metadata)

In [ ]:
metadata_df.reset_index(drop=True).to_feather(f"{base_path}/data/predictionstte_test_230321.feather")

In [ ]:
metadata_df

In [ ]:
endpoints_md

In [ ]:
wide_df = metadata_df["loghs"].apply(pd.Series)
wide_df.columns = endpoints_md.index
wide_df

predictions = metadata_df.merge(wide_df, how="left", left_index=True, right_index=True).drop("loghs", axis=1)

predictions.to_feather(f"{base_path}/data/predictionsttewide_test_230321.feather")